In [18]:
!pip install requests pandas python-dotenv


In [22]:
class Node:
    def __init__(self, type, value=None, left=None, right=None):
        self.type = type  # 'operator' or 'operand'
        self.value = value  # Operand value or operator (e.g., 'AND')
        self.left = left  # Left child node
        self.right = right  # Right child node


In [26]:
def evaluate_rule(ast, user_data):
    if ast.type == 'operand':
        key, condition = list(ast.value.items())[0]

        # Handle string comparisons
        if isinstance(user_data[key], str):
            # Condition should be like "Sales" and not 'Sales'
            condition_value = condition.strip("'")
            return user_data[key] == condition_value

        # Handle numeric comparisons
        return eval(f"{user_data[key]} {condition}")

    elif ast.type == 'operator':
        if ast.value == 'AND':
            return evaluate_rule(ast.left, user_data) and evaluate_rule(ast.right, user_data)
        elif ast.value == 'OR':
            return evaluate_rule(ast.left, user_data) or evaluate_rule(ast.right, user_data)

    return False


In [24]:
import sqlite3

# Initialize SQLite database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS rules (id INTEGER PRIMARY KEY, rule TEXT)")

# Store a rule in the database
def store_rule(rule_string):
    cursor.execute("INSERT INTO rules (rule) VALUES (?)", (rule_string,))
    conn.commit()

store_rule("age > 30 AND department = 'Sales'")


In [27]:
# User data and rule
user_data = {'age': 35, 'department': 'Sales'}
rule = create_rule("age > 30 AND department = 'Sales'")

# Evaluate the rule
print(evaluate_rule(rule, user_data))  # Expected Output: True


True
